In [1]:
# shopee.py
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.options import Options
import time

# Scraping Link Produk

## Fungsi Scraping Link Produk
Fungsi untuk scraping link produk yang sesuai dengan kata kunci

In [2]:

def scrapeLink(list_product, category):
    chrome_options = Options()
    # chrome_options.add_argument('--headless')
    chrome_options.add_argument('log-level=2')
    driver = webdriver.Chrome('chromedriver', options=chrome_options)
    # katakunci = input('Masukkan kata kunci : ')

    def search(katakunci):
        links = []
        print('mencari semua product dengan kata kunci ' + katakunci)
        url = 'https://shopee.co.id/search?keyword=' + katakunci
        try:
            driver.get(url)
            time.sleep(5)
            driver.execute_script('window.scrollTo(0, 1500);')
            time.sleep(5)
            driver.execute_script('window.scrollTo(0, 2500);')
            time.sleep(5)
            soup_a = BeautifulSoup(driver.page_source, 'html.parser')
            products = soup_a.find('div', class_='row shopee-search-item-result__items')
            for link in products.find_all('a'):
                links.append('https://shopee.co.id/' + str(link.get('href')))
                # print(link.get('href'))
        except TimeoutException:
            print('failed to get links with query ' )
        return links
    
    for katakunci in list_product:
        print('Scrape link ' + katakunci)
        product_urls = search(katakunci)
        dict_urls = {'url' : product_urls}
        df = pd.DataFrame(data = dict_urls)
        PATH = '../data/raw/'+ category +'/link_product'
        file_name = PATH + '/links_' + katakunci +'.csv'
        df.to_csv(file_name, index=False)


## Kategori dan Kata Kunci
Menginisiasi list kategori dan kata kunci

In [3]:
CATEGORY = ['fashion', 'food', 'electronic']

fashion = ['hoodie', 'hijab', 'celana', 'kemeja', 'gamis', 'sepatu', 'kaos', 'sandal', 'tas', 'jeans', 'jaket', 'sweater']
food = ['snack', 'daging', 'selai', 'gula', 'susu', 'mie', 'roti', 'bumbu', 'camilan', 'bahan_kue', 'minuman', 'bahan_pokok']
electronic = ['laptop', 'mouse', 'handphone', 'earphone', 'tv', 'mixer', 'setrika', 'mesin_cuci', 'penyedot_debu', 'kamera', 'speaker', 'lampu']


## Scraping Link Produk
Scraping link produk berdasarkan kata kunci

In [4]:
# for category in [fashion, food, electronic]:
#     scrapeLink(category)

# Scraping Ulasan Produk
Scraping ulasan produk dari link yang telah ada

In [5]:
from os import walk
import re
import timeit
import requests

## Fungsi Scraping Ulasan

In [6]:
def shopeeScraper (url):
    url = url
    r = re.search(r'i\.(\d+)\.(\d+)', url)
    shop_id, item_id = r[1], r[2]
    ratings_url = 'https://shopee.co.id/api/v2/item/get_ratings?filter=0&flag=1&itemid={item_id}&limit=20&offset={offset}&shopid={shop_id}&type=0'
    data_scrape = []
    start = timeit.default_timer()
    runtime = 0
    offset = 0
    print('Scraping Process...')
    while True:   
        data = requests.get(ratings_url.format(shop_id=shop_id, item_id=item_id, offset=offset)).json()

        i = 1
        try :
            for i, rating in enumerate(data['data']['ratings'], 1):
                if rating['comment'] == '':
                    pass
                else:
                    data_scrape.append([rating['rating_star'], rating['comment']])
        except :
            pass

        if i % 20:
            break

        stop = timeit.default_timer()
        runtime+= (stop-start)
        print(runtime)
        offset += 20
    print('Scraping Done.')
    stop = timeit.default_timer()
    print('Time: ', stop - start) 
    df = pd.DataFrame(data_scrape, columns=['rating', 'reviews'])
    df = df.dropna(axis=0)
    return df

## List File Link
Membuat list file dari direktori link_product

In [7]:
LINK_PRODUCT = {'fashion' : [], 'food' : [], 'electronic' : []}

for ctg in CATEGORY:
    for (dirpath, dirnames, filenames) in walk('../data/raw/link_product/' + ctg):
        LINK_PRODUCT[ctg].extend(filenames)
        break


## Scraping Ulasan Produk
Scraping Ulasan Produk dari daftar link produk di folder link_product

In [8]:
dataReviews = pd.DataFrame({'rating' : [], 'reviews' : []})
ctg = 'food' # ganti dengan direktori tempat file link
file = LINK_PRODUCT[ctg][0] # ganti index dengan link yang belum di scraping
links = pd.read_csv('../data/raw/link_product/' + ctg + '/' + file)
for i, link in enumerate(links['url']):
    if i >= 0:
        print('Scraping ' + file + ' Link ke-' + str(i))
        reviews = shopeeScraper(link)
        frames = [dataReviews, reviews]
        result = pd.concat(frames).drop_duplicates().reset_index(drop=True)
        dataReviews = result
        if i % 2 == 0:
            dataReviews.to_csv('../data/raw/reviews_product/' + ctg + '/' + file[6:-4] + '_v' + str(i/2) + '.csv', index=False)


Scraping links_susu.csv Link ke-37
Scraping Process...
0.40807210000000005
1.1252001000000007
2.1643468000000023
3.5407984000000035
5.224966900000004
7.230349500000004
9.553966300000006
12.235138000000006
15.205667300000007
18.469510000000007
22.04502520000001
25.922287300000008
30.08424950000001
34.54367660000001
39.300951900000015
44.36532540000002
49.731759400000016
55.40313610000001
61.396745400000015
67.69886570000001
74.29545980000002
81.18007880000002
88.40271960000001
95.94223180000002
103.92776030000002
112.24358650000002
120.87930810000002
129.96555360000002
139.3784845
149.1322499
159.1749145
169.5767612
180.2724345
191.2760998
202.6038156
214.24741089999998
226.20170729999998
238.42167619999998
250.9373513
263.808219
276.9739698
290.41325870000003
304.12929280000003
318.10530300000005
332.38133530000005
346.9419338000001
361.80764990000006
376.9458033000001
392.36075890000006
408.07834320000006
424.08716530000004
440.40820110000004
457.04228430000006
473.99816720000007
491.

In [10]:
LINK_MERGE = []
ctg = 'food' # Ganti dengan kategori data yang ingin digabungkan
for (dirpath, dirnames, filenames) in walk('../data/raw/reviews_product/' + ctg):
        LINK_MERGE.extend(filenames) # Membuat list nama file dari direktori kategori
        break

merge_list, count_data = [], 0
for i in LINK_MERGE:
    data = pd.read_csv('../data/raw/reviews_product/' + ctg + '/' + i)
    count_data += len(data)
    merge_list.append(data)
merge_result = pd.concat(merge_list).drop_duplicates().reset_index(drop=True)
merge_result.to_csv('../data/raw/' + ctg + '_reviews.csv', index=False)

print(count_data)
print(merge_result)

1992261
       rating                                            reviews
0         5.0  Jenis:sp pengembang\nHarga:murah\nKualitas:bai...
1         5.0  Masya Alloh,, pengiriman cepet,, pengemasan ra...
2         5.0  Pengiriman cepat pengemasan cepat hrga murah e...
3         5.0  Harga:Terjangkau\nKualitas:Bagus\nJenis:Pengem...
4         5.0  Jenis:koepoe koepoe baking mix s p pengemulsi\...
...       ...                                                ...
215047    4.0  Harga:baik\nKualitas:baiik\nRasa:baik\n\nBrg S...
215048    5.0  Harga: Murah\nKualitas: Baik\nRasa: Manis\n\nP...
215049    5.0                                        paking rapi
215050    5.0  LalalalaLalalalaLalalalaLalalalaLalalalaLalala...
215051    5.0  Okeeeeee kaaaa\nTerima kasih\nSudah sampai\nAm...

[215052 rows x 2 columns]
